In [2]:
from huggingface_hub import login
login()

In [3]:
from vllm import LLM, SamplingParams

In [4]:


llm = LLM(model='mistralai/Mistral-7B-v0.3', dtype='half') 


WARNING 08-20 15:07:11 config.py:1454] Casting torch.bfloat16 to torch.float16.
INFO 08-20 15:07:11 llm_engine.py:174] Initializing an LLM engine (v0.5.4) with config: model='mistralai/Mistral-7B-v0.3', speculative_config=None, tokenizer='mistralai/Mistral-7B-v0.3', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=mistralai/Mistral-7B-v0.3, use_v2_block_manager=False, enable_prefix_caching=False)
INFO 08-20 15:07:12 selector.py:151] Cannot use FlashAt

/home/kloudvoj/devel/venv/base/lib/python3.11/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/kloudvoj/devel/venv/base/lib/python3.11/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


INFO 08-20 15:07:14 model_runner.py:720] Starting to load model mistralai/Mistral-7B-v0.3...
INFO 08-20 15:07:14 selector.py:151] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 08-20 15:07:14 selector.py:54] Using XFormers backend.
INFO 08-20 15:07:15 weight_utils.py:225] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


INFO 08-20 15:07:50 model_runner.py:732] Loading model weights took 13.5083 GB
INFO 08-20 15:07:56 gpu_executor.py:102] # GPU blocks: 5078, # CPU blocks: 2048
INFO 08-20 15:07:58 model_runner.py:1024] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 08-20 15:07:58 model_runner.py:1028] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 08-20 15:08:14 model_runner.py:1225] Graph capturing finished in 16 secs.


In [58]:

sp = SamplingParams(temperature=0.99, top_p=0.95, max_tokens=128)


In [59]:
MUTATION_PROB = 0.2

def mutate(prompt: str) -> str:
    """ Use LLM to mutate the given prompt; alter it while conserving meaning. """

    instructions = """
                    [INSTRUCTIONS]
                    You are performing a mutation operation as a part of a genetic algorithm. Your task is to mutate and rephrase the given string. 
                    Use fitting synonyms to conserve meaning. Keep the length comparable to the original prompt.
                    [/INSTRUCTIONS]

                    [EXAMPLE]
                    [INPUT]
                    Your mission is to ascertain the sentiment of the Provided text and assign a sentiment label from ['negative', 'positive’].
                    [/INPUT]
                    [OUTPUT]
                    Determine the sentiment of the given sentence and assign a label from ['negative', 'positive'].
                    [/OUTPUT]
                    [/EXAMPLE]

                    [INPUT]
                    {}
                    [/INPUT]

                    [OUTPUT]
                """
    
    output_req = llm.generate(instructions.format(prompt), sampling_params=sp)
    return output_req[0].outputs[0].text.split("[/OUTPUT]")[0]

In [60]:
def crossover(prompt_a: str, prompt_b: str) -> str:
    """ Use LLM to combine important aspects of both parent prompts to generate an 'offspring' prompt. """

    instructions = """
                    [INSTRUCTIONS]
                    You are performing a crossover operation as a part of a genetic algorithm. Your task is to combine string A and string B in a fitting way. 
                    Only add words to connect both parts in a meaningful way. Keep the length comparable to the original prompts.
                    [/INSTRUCTIONS]

                    [EXAMPLE]
                    [INPUT]
                    [A] Now you are a categorizer, your mission is to ascertain the sentiment of the provided text, either favorable or unfavourable. [/A]
                    [B] Assign a sentiment label to the given sentence from ['negative','positive'] and return only the label without any other text. [/B]
                    [/INPUT]
                    [OUTPUT]
                    Your mission is to ascertain the sentiment of the provided text and assign a sentiment label from ['negative', 'positive’]
                    [/OUTPUT]
                    [/EXAMPLE]

                    [INPUT]
                    [A] {} [/A]
                    [B] {} [/B]
                    [/INPUT]

                    [OUTPUT]
                """
    
    output_req = llm.generate(instructions.format(prompt_a, prompt_b), sampling_params=sp)
    return output_req[0].outputs[0].text.split("[/OUTPUT]")[0]


In [99]:
from random import randint
fn = f"evo-iters{randint(0,1000):04}.txt"

In [86]:
import re

NO_NUMBER_IN_SOLUTION_SCORE = 10000

def closest_occuring_num_diff(target: int, solution: str) -> int:
    """ Finds all occuring integers in the solution string a returns the closest one to the target. """
    nums = [int(x) for x in re.findall(r'-?\d+', solution)]
    diffs = [abs(target-x) for x in nums]
    return NO_NUMBER_IN_SOLUTION_SCORE if len(diffs) < 1 else min(diffs)


In [87]:
def selection(solutions: list[str], target: int):
    """ Returns indices of better-performing half of solutions. """
    top_n_solutions = len(solutions) // 2
    sols_and_ix = list(enumerate(solutions))
    sorted_sols = list(sorted(sols_and_ix, key=lambda x: -closest_occuring_num_diff(target, x[1])))
    with open(fn, 'a') as f:
        f.write(f"best score: {-closest_occuring_num_diff(target, sorted_sols[0][1])}")
    return [x[0] for x in sorted_sols[:top_n_solutions]]

In [96]:
def generate_answer(instructions: str, problem: str) -> str:

    prompt = instructions + problem
        
    output_req = llm.generate(prompt, sampling_params=sp)
    ans = output_req[0].outputs[0].text
    return ans

In [89]:
def pop_best_instructions(instructions, best_indices):
    out = []
    for i in sorted(best_indices, reverse=True):
        out.append(instructions.pop(i))
    return out

In [91]:
from random import random, choice

def genetic_algo_step(population: list[str], problem: str, target: int) -> list[str]:
    solutions = []
    for instructions in population:
        solutions.append(generate_answer(instructions, problem))

    best_ix = selection(solutions, target)
    best = pop_best_instructions(population, best_ix)
    
    with open(fn, 'a') as f:
        f.write(best[0])
        f.write('\n ------------------ \n\n')

    crossovers = []
    for ix, solution in enumerate(best):
        other = choice(best[:ix] + best[ix+1:])
        crossovers.append(crossover(solution, other))

    out = best + crossovers

    for ix, solution in enumerate(out):
        if random() < MUTATION_PROB:
            out[ix] = mutate(solution)

    return out


In [92]:
fn

'evo-iters0380.txt'

In [93]:
problem = """
    At the gym, Hillary swims every 6 days, runs every 4 days, and
cycles every 16 days. If she did all three activities today, in how
many days will she do all three activities again on the same
day?
"""
answer = 48

In [97]:
initial_population = [
    "You are a brilliant problem solver! Break it down step by step. First, identify the key numbers and information. Then, use logical reasoning to solve. Remember to check your work for accuracy. Format your answer as an integer. Carefully read this problem:",
    "You love challenges, especially word problems! Start by understanding the scenario, then follow a step-by-step approach to find the solution. Always double-check your calculations. Provide your answer as an integer. Solve this problem: ",
    "You’re great at math, and solving word problems is fun for you! Think carefully about each step, and show your reasoning along the way. Ensure your answer is correct and expressed as an integer. Here's a new challenge:",
    "Get ready to show off your math skills!  Work through the problem logically by identifying key information and solving it step by step. Present your final answer as an integer. Here’s a problem for you:",
    "You’re a math expert and love solving puzzles! Analyze each piece of information, reason through the steps, and perform accurate calculations. Give your answer as an integer. Try this word problem: ",
    "You’re amazing at solving word problems! Think methodically, identify important details, and work step by step to solve. Check your work, and make sure to answer with an integer. Here’s one for you: ",
    "You have a sharp mind for solving problems! Take a thoughtful approach, break the problem down into smaller steps, and carefully calculate your answer. Provide your answer as an integer. Here’s your task:",
    "You enjoy solving word problems, and this one is no exception! Work step by step, use logical reasoning, and make sure your calculations are correct. Your answer should be written as an integer.  Read the problem: ",
    "You’re a great student, and solving problems is your strength! Think through the problem carefully, work through each step, and verify your solution. Your answer should be an integer. Take a look at this:",
    "You have excellent problem-solving skills! Approach it by breaking down each step, use logic to solve it, and be sure to double-check your work. Answer with an integer. Here’s a word problem for you: ",
    "You’re a talented problem solver! Tackle it step by step, reason through the information, and check your calculations. Format your answer as an integer. Check out this word problem: ",
    "You’re a logical thinker and excel at word problems! Break the problem into parts, solve each step carefully, and ensure your calculations are correct. Provide your answer as an integer. Solve this one: ",
    "You’re great at solving math puzzles! Start by understanding the scenario, think through each step, and perform the calculations carefully. Your final answer should be an integer. Try this word problem: ",
    "You’re a brilliant student who loves solving word problems! Take your time, break down each part of the problem, and work your way to the solution. Ensure your answer is an integer. Read this: ",
    "You’re a math whiz! Approach it step by step, solve the problem logically, and verify your work. Answer with an integer. Here's a word problem for you: ",
    "You’re great at solving challenging word problems! Work through it carefully by identifying important details and solving step by step. Remember to give your answer as an integer. Try this one: ",
    "You’re a sharp problem solver!  Think through it logically, break the problem into steps, and solve carefully. Make sure to provide your answer as an integer. Here’s a task for you:",
    "You’re a fantastic problem solver! Approach it step by step, use logical reasoning, and check your calculations. Your answer should be given as an integer. Check out this word problem: ",
    "You’re skilled at solving word problems! Work through the information step by step, think logically, and ensure your calculations are accurate. Write your answer as an integer. Try this one:",
    "You’re great at solving word problems! Break it down into smaller steps, use logical reasoning, and double-check your solution. Answer with an integer. Here’s one for you:"
]

len(initial_population)

20

In [100]:
for step in range(10):
    print(f"{step=}")
    initial_population = genetic_algo_step(initial_population, problem, answer)


with open(fn, 'a') as f:
    f.write('\n ###########################\n ###########################\n ###########################\n \n\n')
    for s in initial_population:
        f.write(s)
        f.write('\n ------------------ \n\n')

step=0














Processed prompts: 100%|██████████| 2/2 [00:03<00:00,  1.69s/it, est. speed input: 62.88 toks/s, output: 75.93 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.15s/it, est. speed input: 49.86 toks/s, output: 41.94 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.04s/it, est. speed input: 34.86 toks/s, output: 42.10 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.14s/it, est. speed input: 88.98 toks/s, output: 41.41 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.02s/it, est. speed input: 99.15 toks/s, output: 41.23 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.04s/it, est. speed input: 34.22 toks/s, output: 42.12 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.04s/it, est. speed input: 33.30 toks/s, output: 42.20 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.04s/it, est. speed input: 33.95 toks/s, ou

step=1










Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.04s/it, est. speed input: 31.97 toks/s, output: 42.19 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.04s/it, est. speed input: 28.97 toks/s, output: 42.14 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.04s/it, est. speed input: 31.58 toks/s, output: 42.11 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.04s/it, est. speed input: 32.54 toks/s, output: 42.08 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.04s/it, est. speed input: 35.18 toks/s, output: 42.09 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.02s/it, est. speed input: 34.82 toks/s, output: 42.12 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.07s/it, est. speed input: 50.22 toks/s, output: 42.01 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.36it/s, est. speed input: 137.52 toks/s, outpu

step=2


...









Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.04s/it, est. speed input: 30.97 toks/s, output: 42.18 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.05s/it, est. speed input: 61.01 toks/s, output: 41.98 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.03s/it, est. speed input: 29.04 toks/s, output: 42.23 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.04s/it, est. speed input: 42.72 toks/s, output: 42.07 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.16s/it, est. speed input: 100.92 toks/s, output: 41.40 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.05s/it, est. speed input: 61.02 toks/s, output: 41.99 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.04s/it, est. speed input: 38.19 toks/s, output: 42.14 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.04s/it, est. speed input: 37.85 toks/s, outpu

step=8










Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.03s/it, est. speed input: 26.40 toks/s, output: 42.24 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.05s/it, est. speed input: 61.02 toks/s, output: 41.99 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.04s/it, est. speed input: 31.98 toks/s, output: 42.20 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.03s/it, est. speed input: 25.40 toks/s, output: 42.22 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.04s/it, est. speed input: 37.87 toks/s, output: 42.15 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.05s/it, est. speed input: 60.97 toks/s, output: 41.96 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  6.46it/s, est. speed input: 763.41 toks/s, output: 32.62 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.04s/it, est. speed input: 28.98 toks/s, outpu

step=9










Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.04s/it, est. speed input: 28.98 toks/s, output: 42.15 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.04s/it, est. speed input: 24.72 toks/s, output: 42.18 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.05s/it, est. speed input: 39.74 toks/s, output: 42.04 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.04s/it, est. speed input: 30.62 toks/s, output: 42.14 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.05s/it, est. speed input: 59.98 toks/s, output: 41.95 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.05s/it, est. speed input: 60.92 toks/s, output: 41.92 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.04s/it, est. speed input: 36.19 toks/s, output: 42.11 toks/s]








Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.06s/it, est. speed input: 60.96 toks/s, output